In [1]:
import pandas as pd
import numpy as np
import obspy
import os
import scipy

In [2]:
# Lunar

l_catalog = pd.read_csv("../data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv")
l_catalog.head(2)

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq


In [ ]:
data_dirs = ["../data/lunar/training/data/S12_GradeA", ]

In [197]:
data    = []
targets = []

# for Lunar
l_catalog = pd.read_csv("../data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv")
l_dir = "../data/lunar/training/data/S12_GradeA"
for i in l_catalog.index:
    filename = f"{l_dir}/{l_catalog.loc[i, 'filename']}.mseed"
    if os.path.exists(filename):
        st = obspy.read(filename)
        f, t, sxx = scipy.signal.spectrogram(st.traces[0].data, st.traces[0].stats.sampling_rate)
        tmp = np.zeros((sxx.shape[1]-4, 1, 5, sxx.shape[0]))
        for k in range(4, sxx.shape[1]-4):
            tmp[k][0] = sxx.T[k-4:k+1]
        data.append(tmp)

        del tmp

        tmp = np.zeros((sxx.shape[1]-4))
        tmp_target_id = l_catalog.loc[i, 'time_rel(sec)'] // st.traces[0].stats.sampling_rate
        tmp_target_id = (tmp_target_id / st.traces[0].data.shape) * sxx.shape[1]
        tmp[int(tmp_target_id+1)] = 1

        targets.append(tmp)

        del tmp
    
    else:
        print(filename)


# for Mars
m_catalog = pd.read_csv("../data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv")
m_dir = "../data/mars/training/data"
for i in m_catalog.index:
    filename = f"{m_dir}/{m_catalog.loc[i, 'filename'][:-4]}.mseed"
    if os.path.exists(filename):
        st = obspy.read(filename)
        f, t, sxx = scipy.signal.spectrogram(st.traces[0].data, st.traces[0].stats.sampling_rate)
        tmp = np.zeros((sxx.shape[1]-4, 1, 5, sxx.shape[0]))
        for k in range(4, sxx.shape[1]-4):
            tmp[k][0] = sxx.T[k-4:k+1]
        data.append(tmp)

        del tmp

        tmp = np.zeros((sxx.shape[1]-4))
        tmp_target_id = m_catalog.loc[i, "time_rel(sec)"] // st.traces[0].stats.sampling_rate
        tmp_target_id = (tmp_target_id / st.traces[0].data.shape) * sxx.shape[1]
        tmp[int(tmp_target_id+1)] = 1

        targets.append(tmp)

        del tmp
    
    else:
        print(filename)

/tmp/ipykernel_6351/2032607069.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp[int(tmp_target_id+1)] = 1


../data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.mseed


/tmp/ipykernel_6351/2032607069.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tmp[int(tmp_target_id+1)] = 1


In [212]:
max_length = max(arr.shape[0] for arr in data)
data_pad = []
targets_pad = []
for arr, tar in zip(data, targets):
    if arr.shape[0] < max_length:
        padding = [(0, max_length - arr.shape[0])] + [(0, 0)] * 3
        padded_arr = np.pad(arr, padding, mode='constant')
        data_pad.append(padded_arr)
    else:
        data_pad.append(arr)
    
    if tar.shape[0] < max_length:
        padding = [(0, max_length - tar.shape[0])]
        padded_tar = np.pad(tar, padding, mode='constant')
        targets_pad.append(padded_tar)
    else:
        targets_pad.append(tar)

In [213]:
data    = np.array(data_pad)
targets = np.array(targets_pad)

In [216]:
data.shape

(77, 2551, 1, 5, 129)

In [214]:
import torch

In [215]:
torch.save(torch.tensor(data   , dtype=torch.float32), "data.pt"   )
torch.save(torch.tensor(targets, dtype=torch.float32), "targets.pt")